<a href="https://colab.research.google.com/github/parthkhambhayta0705/Demos/blob/master/Spark_transformation_and_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
import sys,tempfile,urllib
from pyspark.sql.functions import *

In [8]:
BASE_DIR='/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR,'corona_data.csv')

In [22]:
corona_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv',CORONA_DATA_FILE)

In [23]:
BASE_DIR='/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR,'twitter_data.csv')

In [25]:
twiiter_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv',TWITTER_DATA_FILE)

In [26]:
!ls /tmp

blockmgr-efe149e5-1d34-4215-9730-4c2d038885f8
corona_data.csv
hsperfdata_root
liblz4-java-6170816833513858442.so
liblz4-java-6170816833513858442.so.lck
spark-6043fd8f-f2d3-4473-962c-0334da916c54
spark-ca36cb38-5595-4df5-b0ef-a722142ae753
twitter_data.csv


In [27]:
corona_df = spark.read.csv("/tmp/corona_data.csv", header=True, inferSchema=True)

In [28]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [29]:
corona_df.count()

28143

In [30]:
twitter_df = spark.read.csv("/tmp/twitter_data.csv", header=True, inferSchema=True)

In [31]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [33]:
twitter_df.count()

1000

In [34]:
twitter_df.filter("country='USA'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [35]:
twitter_df.filter("country='USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [37]:
twitter_df.filter("country='USA' and location like 'New%'").show()

+---+----+--------------------+---------------+------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|          location|            entities|           sentiment|country|
+---+----+--------------------+---------------+------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|     New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
|228|null|This is a very co...|baskingntheGlow|     New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|   New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen|      New York, NY|                ['']|{'neg': 0.0, 'neu...|    USA|
|275|null|  The Republican ...|  GenetBataiile|New Hampshire, US

In [38]:
tw_filter_df = twitter_df.filter("location = 'USA'")

In [39]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#156, geo#157, text#158, user#159, location#160, entities#161, sentiment#162, country#163]
+- *(1) Filter (isnotnull(location#160) AND (location#160 = USA))
   +- FileScan csv [_c0#156,geo#157,text#158,user#159,location#160,entities#161,sentiment#162,country#163] Batched: false, DataFilters: [isnotnull(location#160), (location#160 = USA)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), EqualTo(location,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [42]:
tw_filter_df = twitter_df.filter("country='USA' and location like '%New%'")

In [43]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#156, geo#157, text#158, user#159, location#160, entities#161, sentiment#162, country#163]
+- *(1) Filter (((isnotnull(country#163) AND isnotnull(location#160)) AND (country#163 = USA)) AND Contains(location#160, New))
   +- FileScan csv [_c0#156,geo#157,text#158,user#159,location#160,entities#161,sentiment#162,country#163] Batched: false, DataFilters: [isnotnull(country#163), isnotnull(location#160), (country#163 = USA), Contains(location#160, New)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,USA), StringContains(location,New)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [44]:
tw_filter_df.show(5)

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
+---+----+--------------------+---------------+-

In [46]:
tw_filter_df =twitter_df.filter(col('location').startswith('N'))

In [47]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#156, geo#157, text#158, user#159, location#160, entities#161, sentiment#162, country#163]
+- *(1) Filter (isnotnull(location#160) AND StartsWith(location#160, N))
   +- FileScan csv [_c0#156,geo#157,text#158,user#159,location#160,entities#161,sentiment#162,country#163] Batched: false, DataFilters: [isnotnull(location#160), StartsWith(location#160, N)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [48]:
tw_filter_df.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|      country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|          USA|
|  7|null| 📽️Friends, I wi...|        LorseaR|New South Wales, ...|[('Twitter', 'GPE...|{'neg': 0.123, 'n...|    Australia|
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|          USA|
| 36|null| Before #coronavi...|   Frankapistan|       Nashville, TN|[('China', 'GPE')...|{'neg': 0.144, 'n...|          USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|          USA|
|

In [49]:
twitter_df

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [50]:
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [51]:
twitter_df.take(5)

[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [52]:
x = spark.sparkContext.parallelize([1,2,4,8])
y = x.map(lambda x:(x, x*x))
print(x.collect())
print(y.collect())

[1, 2, 4, 8]
[(1, 1), (2, 4), (4, 16), (8, 64)]


In [54]:
x = spark.sparkContext.parallelize([1,2,4,8])
y = x.flatMap(lambda x:(x, x*x))
print(x.collect())
print(y.collect())

[1, 2, 4, 8]
[1, 1, 2, 4, 4, 16, 8, 64]


In [55]:
x

ParallelCollectionRDD[87] at readRDDFromFile at PythonRDD.scala:262

In [57]:
twitter_df.select('user').show()

+--------------------+
|                user|
+--------------------+
|          petodinice|
| but i took the t...|
|              cek422|
|        InfectiousDz|
|          vic_gibson|
|      StewartNgilana|
|           BWheatnyc|
|             LorseaR|
|         straightj23|
|       _______coolio|
|      champagneaylin|
|       YorkLawLondon|
|      BeesonMargaret|
|      harrytiffanyiv|
|         grammyheath|
|      barbara_ellena|
|               fatal|
|          pwjkmiller|
|        Mrrandy123RP|
|           aliensky0|
+--------------------+
only showing top 20 rows



In [58]:
twitter_df.select('text','user').show()

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|What is God sayin...|          petodinice|
|"BREAKING: ""this...| but i took the t...|
| #Coronavirus tes...|              cek422|
| Get ready for ma...|        InfectiousDz|
| The #coronavirus...|          vic_gibson|
| COVID-19 update ...|      StewartNgilana|
| It’s painful to ...|           BWheatnyc|
| 📽️Friends, I wi...|             LorseaR|
| Questions about ...|         straightj23|
|How they’re deali...|       _______coolio|
| BREAKING: Democr...|      champagneaylin|
| “If we close dow...|       YorkLawLondon|
| I pity the poor ...|      BeesonMargaret|
| We’re the heck w...|      harrytiffanyiv|
| I don't think sh...|         grammyheath|
| Well written, so...|      barbara_ellena|
| 1/2 CDC Director...|               fatal|
| In all seriousne...|          pwjkmiller|
| Wash your hands....|        Mrrandy123RP|
| #CoronaVirusCana...|           

In [59]:
twitter_df.rdd.map(lambda l:l.text.split(" ")).take(5)

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [61]:
twitter_df.rdd.flatMap(lambda l: l.text.split(" ")).take(100)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:',
 'cricket',
 'in',
 '#Australia',
 'in',
 'empty',
 'st…',
 '',
 'The',
 '#coronavirus',
 'pandemic',
 'is',
 'revealing',
 'just',
 'how',
 'closely',
 'we',
 'are',
 'all',
 'bound',
 'together...[A',
 'thread]',
 '',
 '',
 'COVID-19',
 'update',
 'as',
 'of',
 'this',
 'morning:1.',
 'Death',
 'toll',
 'in',
 'Italy',
 'passes',
 '1,0002.',
 "Arsenal's",
 'head',
 'coach',
 'Arteta',
 'tests',
 'positive3.',
 'US…',
 '',
 'It’s',
 'painful',
 'to',
 'say,',
 'but',
 'as',
 'an']

In [62]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [67]:
corona_df.filter('country = "US"').show()

+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [70]:
corona_df.filter('country = "US"').sort(col("Date"), ascending = False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [71]:
#Same output of sort and orderBy

corona_df.filter("country = 'US'").orderBy(col("Date"), ascending = False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [77]:
corona_df.filter("country='US'").sortWithinPartitions([col('Date'),col('confirmed')], ascending = False).show()

+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [79]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [80]:
corona_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [82]:
corona_df.filter('confirmed>10000').sort(col('confirmed')).show()

+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18|    12327|   28|      105|       Berlin|null|
|23866|  null|  Italy|   43.0|    12.0|2

In [83]:
corona_df.filter('confirmed>10000').approxQuantile('confirmed',[0.25,0.5,0.75,0.9,0.95],0.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [86]:
#collect give in row formate

corona_df.agg({'Date':'max'}).collect()

[Row(max(Date)='2020-03-20')]

In [94]:
corona_df.agg({'date':'max','confirmed':'max'}).collect()

[Row(max(date)='2020-03-20', max(confirmed)=67800)]

In [91]:
max_date = corona_df.agg({'date':'max'})

In [92]:
max_date.show()

+----------+
| max(date)|
+----------+
|2020-03-20|
+----------+



In [102]:
corona_df.select([max('date'),max('confirmed')]).show()

+----------+--------------+
| max(date)|max(confirmed)|
+----------+--------------+
|2020-03-20|         67800|
+----------+--------------+



In [103]:
import pyspark.sql.functions as F
corona_df.groupBy('Country','state_cleaned').agg(F.max('date')).show()

+--------------+--------------------+----------+
|       Country|       state_cleaned| max(date)|
+--------------+--------------------+----------+
|      Cameroon|             Yaounde|2020-03-20|
|         China|             Qinghai|2020-03-20|
|        Cyprus|             Nicosia|2020-03-20|
|            US|            Michigan|2020-03-20|
|      Portugal|              Lisbon|2020-03-20|
|            US|            Colorado|2020-03-20|
|United Kingdom|      Cayman Islands|2020-03-20|
|         China|              Hainan|2020-03-20|
|            US|            Missouri|2020-03-20|
|     Australia|Australian Capita...|2020-03-20|
|            US|                Guam|2020-03-20|
|        France|             Reunion|2020-03-20|
|      Colombia|        Cundinamarca|2020-03-20|
|          Cuba|              Havana|2020-03-20|
|     Mauritius|          Port Louis|2020-03-20|
|       Ukraine|                Kiev|2020-03-20|
|         Benin|          Porto-Novo|2020-03-20|
|   Switzerland|    

In [104]:
corona_df.join(corona_df.groupBy('Country','state_cleaned').agg(F.max('date').alias('Date')),on=['country','state_cleaned','Date'],how='inner').show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [106]:
corona_df.join(corona_df.groupBy('Country','state_cleaned').agg(F.max('date').alias('Date')),on=['country','state_cleaned','Date'],how='inner').sort('confirmed', ascending = False).show(10)

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|    City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|    null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|    null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|    null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|    null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|               53.0|    

In [109]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

ws=Window().partitionBy("Country","State_cleaned").orderBy(col("Date").desc())

corona_df.withColumn("row_num",row_number().over(ws)).show()

+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cameroon|3.8480000000000003|11.5021|2020-03-15|        2|    0|        0|  

In [110]:
corona_df.withColumn('row_num',row_number().over(ws)).where(col('row_num')==1).show()

+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3266| -84.5361|2020-03-20|      552|    3|        0|   

In [111]:
corona_max_df= corona_df.join(corona_df.groupBy('Country','state_cleaned').agg(F.max('date').alias('Date')),on=['country','state_cleaned','Date'],how='inner')

In [112]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [117]:
corona_df.groupBy('country').pivot('date').agg(F.sum('confirmed')).show()

+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|    country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|2020-02-19|2020-

In [118]:
corona_df.filter('country="US"').crosstab('state','date').show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         state_date|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [119]:
corona_max_df.groupBy('country').agg(F.sum('confirmed')).show()

+-----------+--------------+
|    country|sum(confirmed)|
+-----------+--------------+
|       Chad|             1|
|   Paraguay|            13|
|     Russia|           253|
|    Senegal|            38|
|     Sweden|          1639|
| Cabo Verde|             1|
|     Guyana|             7|
|Philippines|           230|
|   Djibouti|             1|
|  Singapore|           385|
|   Malaysia|          1030|
|       Fiji|             1|
|     Turkey|           359|
|       Iraq|           208|
|    Germany|         19848|
|   Cambodia|            51|
|Afghanistan|            24|
|     Jordan|            85|
|   Maldives|            13|
|     Rwanda|            17|
+-----------+--------------+
only showing top 20 rows



In [120]:
corona_max_df.groupBy('country').agg(F.sum('confirmed'),F.sum('Death'),F.sum('recovered')).orderBy('sum(confirmed)',ascending =False).show()

+--------------+--------------+----------+--------------+
|       country|sum(confirmed)|sum(Death)|sum(recovered)|
+--------------+--------------+----------+--------------+
|         China|         81250|      3253|         71266|
|         Italy|         47021|      4032|          4440|
|         Spain|         20410|      1043|          1588|
|       Germany|         19848|        67|           180|
|          Iran|         19644|      1433|          6745|
|            US|         19100|       244|             0|
|        France|         12726|       450|            12|
|  Korea, South|          8652|        94|          1540|
|   Switzerland|          5294|        54|            15|
|United Kingdom|          4014|       178|            67|
|   Netherlands|          3003|       107|             2|
|       Austria|          2388|         6|             9|
|       Belgium|          2257|        37|             1|
|        Norway|          1914|         7|             1|
|        Swede

In [125]:
corona_df.filter('country="Italy"').sort('Date',ascending = False).show()

+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0|2020-03-13|    17660| 1266|     1439|         Rome|null|
|23866| null|  Italy|43.0|12.0|2

In [127]:
corona_max_df.withColumn('Active',corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort('Active',ascending =False).show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [128]:
corona_max_df = corona_max_df.withColumn('Active',corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death)

In [129]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|Active|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+------+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|   279|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|   739|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|   261|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|     0|
|            Malaysia|             Sarawak|2020-03-20|2

In [130]:
corona_max_df.groupBy('country').sum('active').orderBy('sum(active)', ascending = False).show()

+--------------+-----------+
|       country|sum(active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows



In [135]:
corona_max_df= corona_df.join(corona_df.groupBy('Country','state_cleaned').agg(F.max('date').alias('Date')),on=['country','state_cleaned','Date'],how='inner')

In [136]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [138]:
corona_max_df.select('country','state_cleaned','recovered','confirmed').filter(col('country').isin('Australia','China')).groupBy('country').sum().show(100)

+---------+--------------+--------------+
|  country|sum(recovered)|sum(confirmed)|
+---------+--------------+--------------+
|    China|         71266|         81250|
|Australia|            26|           791|
+---------+--------------+--------------+



In [139]:
corona_max_df.select('country','state_cleaned','recovered','confirmed').filter(col('country').isin('Australia','China')).cube('country').sum().show(100)

+---------+--------------+--------------+
|  country|sum(recovered)|sum(confirmed)|
+---------+--------------+--------------+
|Australia|            26|           791|
|     null|         71292|         82041|
|    China|         71266|         81250|
+---------+--------------+--------------+



In [142]:
corona_max_df.select('country','state_cleaned','recovered','confirmed').filter(col('country').isin('Australia','China')).cube('country','state_cleaned').sum().sort(asc('country')).show(100)

+---------+--------------------+--------------+--------------+
|  country|       state_cleaned|sum(recovered)|sum(confirmed)|
+---------+--------------------+--------------+--------------+
|     null|               Jilin|            92|            93|
|     null|           Hong Kong|            98|           256|
|     null|              Yunnan|           172|           176|
|     null|               Gansu|            98|           134|
|     null|                null|         71292|         82041|
|     null|             Beijing|           390|           491|
|     null|           Guangdong|          1323|          1395|
|     null|          Queensland|             8|           184|
|     null|               Henan|          1250|          1273|
|     null|               Anhui|           984|           990|
|     null|              Hainan|           161|           168|
|     null|               Macau|            10|            17|
|     null|From Diamond Prin...|             0|        

In [144]:
corona_max_df.select('country','state_cleaned','recovered','confirmed').filter(col('country').isin('Australia','China')).rollup('country','state_cleaned').sum().sort(asc('country')).show(100)

+---------+--------------------+--------------+--------------+
|  country|       state_cleaned|sum(recovered)|sum(confirmed)|
+---------+--------------------+--------------+--------------+
|     null|                null|         71292|         82041|
|Australia|                null|            26|           791|
|Australia|            Tasmania|             3|            10|
|Australia|From Diamond Prin...|             0|             0|
|Australia|            Victoria|             8|           121|
|Australia|     New South Wales|             4|           353|
|Australia|          Queensland|             8|           184|
|Australia|     South Australia|             3|            50|
|Australia|Australian Capita...|             0|             6|
|Australia|  Northern Territory|             0|             3|
|Australia|   Western Australia|             0|            64|
|    China|           Chongqing|           570|           576|
|    China|            Shandong|           748|        

In [145]:
corona_max_df.corr('confirmed','recovered')

0.8052934236742176

In [146]:
corona_max_df.cache()

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string]

In [147]:
%timeit corona_max_df.count()

The slowest run took 25.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 71 ms per loop


In [148]:
from pyspark import StorageLevel
corona_max_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string]

In [150]:
%timeit corona_max_df.count()

10 loops, best of 3: 55.9 ms per loop


In [151]:
pd = corona_max_df.toPandas()

In [154]:
pd.corr()

,_c0,Lat,Long,Confirmed,Death,Recovered
_c0,1.000000,-0.094019,-0.312907,-0.136334,-0.100278,-0.045917
Lat,-0.094019,1.000000,-0.383389,0.050988,0.034211,0.003501
Long,-0.312907,-0.383389,1.000000,0.128514,0.092183,0.113290
Confirmed,-0.136334,0.050988,0.128514,1.000000,0.926594,0.805293
Death,-0.100278,0.034211,0.092183,0.926594,1.000000,0.660208
Recovered,-0.045917,0.003501,0.113290,0.805293,0.660208,1.000000


In [155]:
corona_max_df.createOrReplaceTempView('corona')

In [157]:
spark.sql('select * from corona').show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [159]:
spark.sql('select * from corona where country in ("Australia","Canada") order by country').show()

+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|  Country|       state_cleaned|      Date|  _c0|               State|               Lat|     Long|Confirmed|Death|Recovered|City|
+---------+--------------------+----------+-----+--------------------+------------------+---------+---------+-----+---------+----+
|Australia|     New South Wales|2020-03-20|27672|     New South Wales|          -33.8688| 151.2093|      353|    6|        4|null|
|Australia|            Victoria|2020-03-20|27673|            Victoria|          -37.8136| 144.9631|      121|    0|        8|null|
|Australia|   Western Australia|2020-03-20|27715|   Western Australia|          -31.9505| 115.8605|       64|    1|        0|null|
|Australia|          Queensland|2020-03-20|27674|          Queensland|          -28.0167|    153.4|      184|    0|        8|null|
|Australia|            Tasmania|2020-03-20|27727|            Tasmania|          -41

In [160]:
spark.sql("select country,state_cleaned,sum(confirmed), sum(recovered) from corona where country in('Australia','Canada') group by rollup (country,state_cleaned) order by country").show()

+---------+--------------------+--------------+--------------+
|  country|       state_cleaned|sum(confirmed)|sum(recovered)|
+---------+--------------------+--------------+--------------+
|     null|                null|          1734|            35|
|Australia|          Queensland|           184|             8|
|Australia|     New South Wales|           353|             4|
|Australia|Australian Capita...|             6|             0|
|Australia|   Western Australia|            64|             0|
|Australia|            Tasmania|            10|             3|
|Australia|            Victoria|           121|             8|
|Australia|                null|           791|            26|
|Australia|  Northern Territory|             3|             0|
|Australia|From Diamond Prin...|             0|             0|
|Australia|     South Australia|            50|             3|
|   Canada|            Manitoba|            17|             0|
|   Canada|             Alberta|           146|        

In [161]:
spark.sql("select country,state_cleaned,sum(confirmed), sum(recovered) from corona where country in('Australia','Canada') group by cube (country,state_cleaned) order by country").show()

+-------+--------------------+--------------+--------------+
|country|       state_cleaned|sum(confirmed)|sum(recovered)|
+-------+--------------------+--------------+--------------+
|   null|       New Brunswick|            11|             0|
|   null|     New South Wales|           353|             4|
|   null|            Victoria|           121|             8|
|   null|         Nova Scotia|            15|             0|
|   null|Australian Capita...|             6|             0|
|   null|      Grand Princess|            10|             0|
|   null|     South Australia|            50|             3|
|   null|          Queensland|           184|             8|
|   null|        Saskatchewan|            20|             0|
|   null|Newfoundland and ...|             4|             0|
|   null|   Western Australia|            64|             0|
|   null|            Tasmania|            10|             3|
|   null|  Northern Territory|             3|             0|
|   null|    British Col